In [1]:
import os 
import json
import pandas as pd
import numpy as np
import time
import velopix_tracking as velopix

from event_model.event_model import event
from algorithms.track_following import track_following
from validator.validator_lite import validate_print

In [2]:
events = []
py_events = []
json_events = []
n_files = 100

for i in range(0, n_files):
    if i == 51:
        """
        There's an issue with event 51 -> module_prefix_sum contains value 79 twice resulting in and indexing error when loading the event
        """
        print(f"Skipping problematic file: velo_event_{i}.json")
    else:    
        print(f"Loading file: velo_event_{i}.json")
        event_file = open(os.path.join("DB/raw", f"velo_event_{i}.json"))
        json_data = json.loads(event_file.read())
        events.append(velopix.Event(json_data))
        py_events.append(event(json_data))
        json_events.append(json_data)
        event_file.close()

Loading file: velo_event_0.json
Loading file: velo_event_1.json
Loading file: velo_event_2.json
Loading file: velo_event_3.json
Loading file: velo_event_4.json
Loading file: velo_event_5.json
Loading file: velo_event_6.json
Loading file: velo_event_7.json
Loading file: velo_event_8.json
Loading file: velo_event_9.json
Loading file: velo_event_10.json
Loading file: velo_event_11.json
Loading file: velo_event_12.json
Loading file: velo_event_13.json
Loading file: velo_event_14.json
Loading file: velo_event_15.json
Loading file: velo_event_16.json
Loading file: velo_event_17.json
Loading file: velo_event_18.json
Loading file: velo_event_19.json
Loading file: velo_event_20.json
Loading file: velo_event_21.json
Loading file: velo_event_22.json
Loading file: velo_event_23.json
Loading file: velo_event_24.json
Loading file: velo_event_25.json
Loading file: velo_event_26.json
Loading file: velo_event_27.json
Loading file: velo_event_28.json
Loading file: velo_event_29.json
Loading file: velo_e

In [3]:
eval_results = []
slope, tol, scat = 1, 1, 1

init_time = []
solver_time = []
validation_time = []
t_total = []
# Initialize solver with current parameters (assumed to take tuples for slopes and tolerances)
for i, event in enumerate(py_events):
    t_start_act = time.time()
    solver = track_following((slope, slope), (tol, tol), scat)
    init_time.append(time.time() - t_start_act)
    t_start = time.time()
    tracks = solver.solve(event=event)
    solver_time.append(time.time() - t_start)
    t_start = time.time()
    validate_print([json_events[i]], [tracks])
    validation_time.append(time.time() - t_start)
    t_total.append(time.time() - t_start_act)

print(f"init: {sum(init_time)} \n solver: {sum(solver_time)} \n validation: {sum(validation_time)} \n total: {sum(t_total)}")

Instantiating track_following solver with parameters
 max slopes: (1, 1)
 max tolerance: (1, 1)
 max scatter: 1

316 tracks including      117 ghosts ( 37.0%). Event average  37.0%
              velo :      172 from      233 ( 73.8%,  73.8%)       11 clones (  6.40%), purity: ( 93.67%,  93.67%),  hitEff: ( 86.42%,  86.42%)
              long :       66 from       79 ( 83.5%,  83.5%)        6 clones (  9.09%), purity: ( 93.50%,  93.50%),  hitEff: ( 85.68%,  85.68%)
         long>5GeV :       36 from       46 ( 78.3%,  78.3%)        2 clones (  5.56%), purity: ( 94.28%,  94.28%),  hitEff: ( 85.82%,  85.82%)
      long_strange :        3 from        3 (100.0%, 100.0%)        0 clones (  0.00%), purity: ( 93.33%,  93.33%),  hitEff: ( 85.71%,  85.71%)
        long_fromb :        3 from        4 ( 75.0%,  75.0%)        0 clones (  0.00%), purity: ( 90.91%,  90.91%),  hitEff: ( 93.33%,  93.33%)
   long_fromb>5GeV :        2 from        3 ( 66.7%,  66.7%)        0 clones (  0.00%), purity: ( 8

In [4]:
eval_results = []
slope, tol, scat = 1, 1, 1
# Initialize solver with current parameters (assumed to take tuples for slopes and tolerances)
t_start_act = time.time()
solver = velopix.TrackFollowing((slope, slope), (tol, tol), scat)
init_time = time.time() - t_start_act
t_start = time.time()
tracks = solver.solve_parallel(events=events)
solver_time = time.time() - t_start
t_start = time.time()
result = velopix.validate_to_json(json_events, tracks, verbose=False)
validation_time = time.time() - t_start
t_total = time.time() - t_start_act

print(f"init: {init_time} \n solver: {solver_time} \n validation: {validation_time} \n total: {t_total}")

Instantiating track_following solver with parameters
 max slopes: (1.0, 1.0)
 max tolerance: (1.0, 1.0)
 max scatter: 1

init: 6.985664367675781e-05 
 solver: 7.592046022415161 
 validation: 4.6336798667907715 
 total: 12.225845575332642


# Runtime optimalisation

| Operation             | Python Implementation [sec] | Rust Baseline Time [sec] | Improved Time [sec]| % Speed Increase|
|-----------------------|-----------------------|--------------------|---------------|------------------|
| `init TrackFollowing` | 7.25e-04              | 8.82e-05           | 6.29e-05      | 1053.5%          |
| `solver`              | 89.32                 | 7.74               | 7.82          | 1042.2%          |
| `validate`            | 16.07                 | 28.84              | 4.58          | 250.6%           |
| **Total**             | **105.40**            | **36.59**          | **12.23**     | 761.8%           |

